# ABAQUS

Write an abaqus input to calculate stresses related variations in gravitational potential energy. 
The abaqus input file created by this script has the following features:

- Geometry: 
    - 2 spherical layers of thick shell continuum elements. 
    - Top layer thickness is specified by user and bottom layer has a thickness of 100 km. 
    - Bottom layer of nodes are pinned (i.e. fixed degrees of freedom in u1, u2 and u3 directions).  
    - The spacing of the grid is defined by the user and the grid geometry files are read in from external files.

- Materials: 
    - Linearly elastic solid.  The poisson's ratio is 0.3 in both layers.  
    - The Young's modulus of the base layer is low (1e6) in order to prevent transmission of stresses related to the pinned basal nodes.  
    - The upper layer has a uniform Young's modulus of 1e11 or a variable Young's modulus.  
    - For the variable Young's modulus case, the user specifies a separate Young's modulus value for plate interior and plate boundary regions (plate boundary regions should be weaker). 
    - The plate boundaries are defined by either world strain map active deforming zones or the nuvel0a plate boundaries.

- Loads: 
    - Outward tractions (GPE/thickness) applied to the horizontal faces of each element.  
    - The direction of the traction vector will be determined by the sign of the outward tractions. 
    - Note that outward tractions are read in from an external file.  An option exists to restrict mean mean outward pressure variations to specific geographic regions.

- Displacements: 
    - The code also will write an input file that displaces the nodes of the upper element layer in accordance with estimates of dynamic topography.  
    - Note that this option cannot be used at the same time as the GPE loading option as in this case a strain is being applied (the GPE loads are an applied stress).

Last Modified: August 24, 2021. by Gan Kuhasubpasin


## Import

In [1]:
import numpy as np
import pandas as pd
import math 
import scipy.interpolate
from matplotlib import cm
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

/Users/boontigan/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 1. Specifice analysis type and Loading input

In [16]:
analysis_type = int(input('Enter analysis type: 1 for GPE, 2 for dynamic radial displacement  '))
resolution = int(input('Enter grid resolution (2, 1, 0.5 or 0.25 degrees) '))
if analysis_type == 1:
    root = input('Enter root name of geom and gpe files (root_GPE/GEOM) ')
    
    # Loading interpolated GPE pressure file
    gpeFile = root+'_GPE_EA'+str(resolution)+'E'
    PRS = np.loadtxt(gpeFile)
    
elif analysis_type == 2:
    dynfile = input('Enter name of dynamic topography file on EA nodal grid')
    root = input('Enter root name for input file (root.inp) ')
    dynfiledirec = '../LDS_Data/LDS_Data_DynTopo/'+dynfile
    dyntopo = np.loadtxt(dynfiledirec)


Enter analysis type: 1 for GPE, 2 for dynamic radial displacement  1
Enter grid resolution (2, 1, 0.5 or 0.25 degrees) 2
Enter root name of geom and gpe files (root_GPE/GEOM) test


FileNotFoundError: test_GPE_EA2E not found.

## 2. Specifications for model size, geometry and rheology

In [3]:
# Defining the number of shared edges
nelsEdge = 2*math.ceil(54/(2*resolution))

# Calculating the number of spherical elements and nodes
sphElements = nelsEdge*nelsEdge*12; 
sphNodes = ((nelsEdge-1)*(nelsEdge-1)*12) + 24*(nelsEdge+1)-34

# Define the number of lithospheric layers (it should be 1 when using continuum shell elements)
numLayers = 1

#User defines the thickness of the lithospheric shell element layer.
LithLayerThk = int(input('Enter the thickness (km) of the lithosphere element layer  '))

# Defining the thickness of the weak base layer element thickness.
baseLayerThk = 100

# Specify Poisson's ratio for upper layer and basal layer.
Poisson = np.zeros(3)
Poisson[0] = 0.3  # upper layer (plate interior)
Poisson[1] = 0.3  # upper layer (plate boundary)
Poisson[2] = 0.3  # basal layer

# Upper (stong) layer rheology.  The user can choose to have a constant Young's modulus value for the top layer, which would be defined by the  variable topE (topE is always set to 1E11).  
# Alternatively, the user can choose to have elements within plate boundary regions be weaker (lower Young's modulus) than the plate interiors.  
# In this case the user defines the Young's  modulus value for the plate boundary region elements (PBtopE) and the Young's modulus for the plate interior (PItopE) is set to 1E11.

topEinput = int(input('Enter 1 for uniform top layer Youngs Modulus, 2 for weak plate boundaries'))

Young = np.zeros(3)
if topEinput == 1:
    Young[0] = 1e11
    Young[1] = 1e11
else:
    Young[0] = 1e11
    Young[1] = float(input('Enter Youngs Modulus of top layer elements in plate boundary regions'))

# Basal layer Young's modulus
Young[2] = 1e6

Enter the thickness (km) of the lithosphere element layer  100
Enter 1 for uniform top layer Youngs Modulus, 2 for weak plate boundaries1


## 3. Loading file for creating Abaqus mesh

In [4]:
# Loading file that shows which nodes are associated with each element
eco_file = 'EA_ECon_' + str(resolution) + '.txt'
EC = np.loadtxt(eco_file, delimiter=',')

# Loading file that contains the node number and 3 cartesian coordinates 
# defining the node location relative to a sphere with a non-dimensional 
# radius of 1.
nds_file = 'EA_Nodes_' + str(resolution) + '.txt'
N = np.loadtxt(nds_file, delimiter=',')

# Loading file that contains the element number and 3 cartesian 
# coordinates defining the center of each element relative to a sphere 
# with a non-dimensional radius of 1.
els_file = 'EA_Elements_' + str(resolution) + '.txt'
E = np.loadtxt(els_file, delimiter=',')

# Loading files that indicate the elements within the specified distance of the plate boundary, 
# if the weak plate boundary element option was selected.
if topEinput == 2  :
    # User chooses what file to use for distinguishing plate boundary vs. plate interior elements.  
    # Columns in the plate boundary and plate interior files:
    #   1) element number  2) longitude ( 0 - 360)  3) latitude (-90 - 90) 
    filePBname = input('Enter element plate boundary file name  ')
    filePB = np.loadtxt(filePBname, delimiter=',')
    
    filePIname = input('Enter element plate interior file name  ')
    filePI = np.loadtxt(filePIname, delimiter=',')


## 4. Section to specificy options for applying mean MOP in GPE case



In [5]:
if analysis_type == 1:

    # User chooses whether or not to restrict geographical region where 
    # mean outward pressure values are applied.  Outside of the specified 
    # geographical region mean outward pressure values are set to the 
    # reference column MOP value
    restrictMOP = int(input('Enter 1 to restrict MOP values to a specific geographical region, 0 for no'))

    # If option selected, specify lat and lon of region where MOP values will be applied
    if restrictMOP == 1:
        print('Think carefully about how the restricted MOP region lat-lon values are assigned!')
        print('Latitude ~ -90 - 90;  Longitude ~ -180 - 180')
        latMax = float(input('Enter maximum latitude of region with non-zero MOP values'))
        latMin = float(input('Enter minimum latitude of region with non-zero MOP values'))
        lonMax = float(input('Enter maximum longitude of region with non-zero MOP values'))
        lonMin = float(input('Enter minimum longitude of region with non-zero MOP values'))

    # Create a new variable containing mean outward pressure values from column 6 of the GPE file (PRSvariable).  
    # The MOP value of each element outside the specified geographical area is set to the
    # MOP value of the reference column.
    if restrictMOP == 1:
        PRS_Restricted = np.zeros(sphElements)
        for i in range(0,sphElements):
            if PRS[i,1]<=latMax and PRS[i,1]>=latMin and PRS[i,2]<=lonMax and PRS[i,2]>=lonMin:
                PRS_Restricted[i] = PRS[i,4]
            else:
                PRS_Restricted[i] = PRS[5559-1,4]


Enter 1 to restrict MOP values to a specific geographical region, 0 for no0


## 5. Name abaqus input file and open it to write

In [6]:
res_string = '_EA' + str(resolution)

#% If GPE loading analysis selected
if analysis_type == 1:

    # String for models with MOP values restricted to a specifc geographic region
    if restrictMOP==1:
        # Create string that describes geographical restrictions for MOP values 
        
        #Min latitude
        if latMin >= 0:
            latMin_string = str(latMin)+'N'
        elif latMin < 0:
            latMin_string = str(latMin)+'S'
         
        # Max latitude
        if latMax >= 0:
            latMax_string = str(latMax) + 'N'
        elif latMax < 0:
            latMax_string = str(abs(latMax)) + 'S'
        
        # Min longitude
        if lonMin >= 0:
            lonMin_string = str(lonMin) + 'E'
        elif lonMin < 0:
            lonMin_string = str(abs(lonMin)) + 'W'
        
        # Max longitude
        if lonMax >= 0:
            lonMax_string = str(lonMax)+'E'
        elif lonMax < 0:
            lonMax_string = str(abs(lonMax))+'W'
        
        # String
        restrictMOP_string = '_R'+latMin_string+latMax_string+lonMin_string+lonMax_string
    elif restrictMOP==0:
        restrictMOP_string = ''

    # String for models with lower Young's modulus at weak plate boundaries
    if topEinput == 2:
        if len(filePBname) == 23:
            wpb_string = '_WPB' + str(np.log10(Young[0]/Young[1]))+'WSM'
        elif len(filePBname) == 26:
            wpb_string = '_WPB'+str(np.log10(Young[0]/Young[1]))+'nuvel0'
    
    elif topEinput == 1:
        wpb_string = ''

    # Abaqus file name
    abqfile = root + res_string + restrictMOP_string + wpb_string +'.inp'

elif analysis_type == 2:

    # Abaqus file name
    abqfile = root + res_string + '.inp'    
    
# Opening the output file (i.e. abaqus input file)    
fidab = open(abqfile, 'w')


## 6. Write file with restricted MOP values

In [7]:
if analysis_type == 1:
    if restrictMOP == 1:
        restrictMOPfile = root + res_string + restrictMOP_string + '_MOP'
        fid = open(restrictMOPfile,'w')
        for i in range(0,sphElements):
            fid.write('{}\t{}\t{}\n'.format(PRS[i,2],PRS[i,1],PRS_Restricted[i]))
        fid.close()

## 7. Give dimensional values for the size of the mesh

In [8]:
# In this section the dimensional values for the size of the mesh are given (i.e. dimensional radius).  
# In addition, if the user requested to do so, 
# the surface nodes are displaced according to the elevation located in the geometry file.

#  Construct radial geometry of mesh.

# defining the radius for the sphere in km
outerRad = 6371

# Defining the a starting surface radius of 6371 km.  
# If the user requested varibale surface, 
# the elevation will be added to the surface radius a few steps ahead.
R = np.zeros((sphNodes,numLayers+2))
R[:,0] = outerRad + np.zeros(sphNodes)

# For the lithospheric layer subtract the layer thickness from a surface radius of 6371.
for i in range(1,numLayers+1):
    R[:,i] = R[:,i-1] - LithLayerThk

# Subtract the radius of the base layer.  This gives the radius at the base of the weak elements.
R[:,numLayers+1] = R[:,numLayers] - baseLayerThk

# Convert R to meters**
R = R * 1000;

# Define the number of total element layers for use later.
totalELlayers = len(R[0]) - 1


## 8. Write Input File

## 8.1 Write header, node and element

In [9]:
# Beginning to Write Input File --> writing element section (element and associated node numbers) 
# and node number + cartesian coordinates of each node.

# write file heading
fidab.write('*HEADING\n{}\n**\n'.format(abqfile))

# Assign nodes to elements, i.e. "element" section (2 element layers). 
# Note that here the way in which the nodes are assigned to the element 
# is different than in the element connectivity file.  
# Here, the first four nodes defined are for the bottom of the element 
# and the ordering is clock-wise around the element for those four nodes.  
# The top layer of 4 nodes, defined second, are also ordered clockwise around the element.  
# This new ordering scheme is required in order to be consistent with the abaqus shell 
# stress in-plane coordinate system, which depeneds on the way the nodes are assigned to the element!

#fprintf('Creating Elements\n\n');
for elelay in range(numLayers+1):
    fidab.write('*ELEMENT, TYPE=SC8R, ELSET=Layer_{}_Els\n'.format(elelay+1))
    for i in range(sphElements): 
        element = EC[i,0]
        node = EC[i,1:9]          
        fidab.write(' %10d' %(element+(sphElements*(elelay))))
        # first bottom node
        fidab.write(', %10d' %(node[4]+(sphNodes*(elelay))))
        # second bottom node
        fidab.write(', %10d' %(node[7]+(sphNodes*(elelay))))
        # third bottom node
        fidab.write(', %10d' %(node[6]+(sphNodes*(elelay))))
        # fourth bottom node
        fidab.write(', %10d' %(node[5]+(sphNodes*(elelay))))
        # first top node
        fidab.write(', %10d' %(node[0]+(sphNodes*(elelay))))
        # second top node
        fidab.write(', %10d' %(node[3]+(sphNodes*(elelay))))
        # third top node
        fidab.write(', %10d' %(node[2]+(sphNodes*(elelay))))
        # fourth top node
        fidab.write(', %10d' %(node[1]+(sphNodes*(elelay))))
       # fidab.write('%10d %10d %10d %10d %10d %10d %10d %10d %10d \n' %(element+(sphElements*(elelay)),(node[4]+(sphNodes*(elelay))),(node[7]+(sphNodes*(elelay))),(node[6]+(sphNodes*(elelay))),(node[5]+(sphNodes*(elelay))),(node[0]+(sphNodes*(elelay))),(node[3]+(sphNodes*(elelay))),(node[2]+(sphNodes*(elelay))),(node[1]+(sphNodes*(elelay)))))
        fidab.write('\n')

# Write "node" section (3 node layers)
for nodlay in range (0,numLayers+2):
    fidab.write('*NODE, SYSTEM=R, NSET=Layer_{}_Nds\n'.format(nodlay+1))
    for i in range(0,sphNodes):
        fidab.write(' %10d, %12.4e, %12.4e, %12.4e\n' %(N[i,0]+(nodlay)*sphNodes, N[i,1]*R[i,nodlay], N[i,2]*R[i,nodlay], N[i,3]*R[i,nodlay]))


## 8.2 Write element and node sets

In [10]:
# Define sets of elements.  The are two options for defining the element sets:  
# 1) In a given element layer all the material properties are the same
# 2) Elements material properties in a given layer depend on proximity to plate boundaries.

#disp('Writing element set and node set definitions to the input file');

# Define element sets for if material properties do not depend on proximity to plate boundaries.

if topEinput == 1:
    
    # Define element set for lithospheric elements
    fidab.write('*ELSET, ELSET=lithEls\n')
    for i in range(0,numLayers):
        fidab.write('Layer_{}_Els, '.format(i+1))
    fidab.write('\n')

    # Define element set for base elements
    fidab.write('*ELSET, ELSET=baseEls\n')
    i = totalELlayers
    fidab.write('Layer_{}_Els, '.format(i))
    fidab.write('\n')
    
    # Define element set for all elements
    fidab.write('*ELSET, ELSET=allEls\n')
    for i in range(totalELlayers):
        fidab.write('Layer_{}_Els, '.format(i+1))
    fidab.write('\n')

# Define element sets for case where material properties depend on 
# proximity to plate boundaries.
elif topEinput == 2:
    
    # Define element set for lithospheric elements near plate boundaries
    fidab.write('*ELSET, ELSET=lithEls_PB\n')
    for i in range(0,numLayers):
        fidab.write('{}\n'.format(filePB[:,0] + ((i-1)*34992) ))

    # Define element set for lithospheric elements in plate interior
    fidab.write('*ELSET, ELSET=lithEls_PI\n')
    for i in range(0,numLayers):
        fidab.write('{}\n'.format(filePI[:,0] + ((i-1)*34992) ))
    
    # Define element set for base elements
    fidab.write('*ELSET, ELSET=baseEls\n')
    i = totalELlayers
    fidab.write('Layer_{}_Els, '.format(i))
    fidab.write('\n')
    

    # Define element set for all elements
    fidab.write('*ELSET, ELSET=allEls\n')
    for i in range (0,totalELlayers):
        fidab.write('Layer_{}_Els, '.format(i))
    fidab.write('\n')


# Define sets of nodes
fidab.write('*NSET, NSET=botNodes\n')
i = totalELlayers+1;
fidab.write('Layer_{}_Nds, '.format(i))
fidab.write('\n')
fidab.write('**\n')

3

## 8.3 Write assemebly section --> define materials and associated different materials with different elements sets

In [11]:
# Note that thickness values in km are converted to meters for definition in abaqus (i.e. *1000).

# Section to define material properties and solid section for when the 
# rheology does or does not depend on the proximity to the plate boundaries.
if topEinput == 1:

    # Begin Define materials
    fidab.write('** MATERIALS\n')
    fidab.write('**\n')

    # Define material for lithosphere elements
    fidab.write('*MATERIAL, NAME=lithosphere\n')
    fidab.write('*ELASTIC\n')
    fidab.write(' %16.8e, %8.4f\n'%(Young[0], Poisson[0]))
    fidab.write('**\n')

    # Define material for weak base elements
    fidab.write('*MATERIAL, NAME=asthenosphere\n')
    fidab.write('*ELASTIC\n')
    fidab.write(' %16.8e, %8.4f\n'%(Young[2], Poisson[2]))
    fidab.write('**\n')

    # Define shell section for lithospheric 
    fidab.write('*SHELL GENERAL SECTION,ELSET=lithEls,MATERIAL=lithosphere\n')
    fidab.write('{}\n'.format(LithLayerThk*1000))
    
    # Define shell section for weak base elements
    fidab.write('*SHELL GENERAL SECTION,ELSET=baseEls,MATERIAL=asthenosphere\n')
    fidab.write('{}\n'.format(baseLayerThk*1000))

elif topEinput == 2:
    
    # Begin Define materials
    fidab.write('** MATERIALS\n')
    fidab.write('**\n')

    # Define material for plate interior lithospheric elements
    fidab.write('*MATERIAL, NAME=lithosphere_PI\n')
    fidab.write('*ELASTIC\n')
    fidab.write('{}\t{}\n'.format(Young[0], Poisson[0]))
    fidab.write('**\n')
    
    # Define material for plate boundary lithospheric elements
    fidab.write('*MATERIAL, NAME=lithosphere_PB\n')
    fidab.write('*ELASTIC\n')
    fidab.write('{}\t{}\n'.format(Young[1], Poisson[1]))
    fidab.write('**\n')

    # Define material for weak base elements
    fidab.write('*MATERIAL, NAME=asthenosphere\n')
    fidab.write('*ELASTIC\n')
    fidab.write('{}\t{}\n'.format(Young[2], Poisson[2]))
    fidab.write('**\n')

    # Define shell section for plate boundary lithospheric elements
    fidab.write('*SHELL GENERAL SECTION,ELSET=lithEls_PB,MATERIAL=lithosphere_PB\n');
    fidab.write('{}\n'.format(LithLayerThk*1000))
    
    # Define shell section for plate interior lithospheric elements
    fidab.write('*SHELL GENERAL SECTION,ELSET=lithEls_PI,MATERIAL=lithosphere_PI\n');
    fidab.write('{}\n'.format(LithLayerThk*1000))    
    
    # Define shell section for weak base elements
    fidab.write('*SHELL GENERAL SECTION,ELSET=baseEls,MATERIAL=asthenosphere\n');
    fidab.write('{}\n'.format(baseLayerThk*1000))

## 8.4 Write conditions --> basal nodes are pinned

In [12]:
fidab.write('*STEP\n')
fidab.write('*STATIC\n')
fidab.write('*BOUNDARY,OP=NEW\n')
fidab.write('botNodes, 1, 3\n')

15

In [13]:
# Specify dynamic topography nodal displacements if requested by user
if analysis_type == 2:

    # Apply displacement boundary conditions to all nodes in the upper 
    # element layer (8 nodes per element) according to the computed values 
    # of dynamic topography at each nodes.
    # Write first line of displacement boundary condtions
    
    fidab.write('*BOUNDARY,OP=NEW,TYPE=DISPLACEMENT\n')
    # Loop through both nodal layers of the top element layer
    for j in range(0,2):
        # Loop through all of the nodes
        for i in range(0,sphNodes):

            # Read in values of dynamic topography at each lat and lon
            dynPhi = dyntopo[i,1]  # longitude
            dynTheta = dyntopo[i,0]  # latitude
            dynVal = dyntopo[i,2]  # value of dynamic topography in meters

            # Convert angles to radians + convert lat to colat
            if dynPhi < 0:
                dynPhi = dynPhi + 360
            dynPhi = dynPhi*(pi/180)
            dynTheta = (90 - dynTheta)*(pi/180)

            # Transfer dynamic topography values to cartesian coordinates
            x = dynVal*np.sin(dynTheta)*np.cos(dynPhi)
            y = dynVal*np.sin(dynTheta)*np.sin(dynPhi)
            z = dynVal*np.cos(dynTheta)

            # Write displacement boundary conditions to input file
            fidab.write('   {}\t{}\t{}\t{}\n'.format(i+((j-1)*sphNodes),1,1,x)) 
            fidab.write('   {}\t{}\t{}\t{}\n'.format(i+((j-1)*sphNodes),2,2,y)) 
            fidab.write('   {}\t{}\t{}\t{}\n'.format(i+((j-1)*sphNodes),3,3,z)) 

## 8.5 Write the GPE loads into the input file if the GPE analysis was requested by the user.

In [14]:
if analysis_type == 1:

    fidab.write( '**\n')
    fidab.write( '*DLOAD\n')

    subPres=1

    # If subPres = 1 use MOP - MOP_ReferenceColumn (column six of GPE_EA file)
    if subPres==1 and restrictMOP==0:

        for k in range(0,numLayers):
            for i in range(0,sphElements):
                for j in range(3,7):
                    fidab.write('%10d, P%1d, %12.7e\n'%(PRS[i,0]+(sphElements*(k)), j, PRS[i,5]))

    # If subPres = 1 use MOP (column six of GPE_EA file ~ Reference MOP subtracted) 
    # and MOP values are restricted to a specific geographical region.
    elif subPres==1 and restrictMOP==1:

        for k in range(0,numLayers):
            for i in range(0,sphElements):
                for j in range(3,7) :
                    fidab.write('{}\t{}\t{}\n'.format(PRS[i,0]+(sphElements*(k)), j, PRS_Restricted[i]))

    # If subPres = 0 use MOP (column five of GPE_EA file ~ Reference MOP not subtracted) 
    # and MOP values are not restricted to a specific geographical region.
    elif subPres==0 and restrictMOP==0:
        for k in range(0,numLayers):
            for i in range(0,sphElements):
                for j in range(3,7) :
                    fidab.write('{}\t{}\t{}\n'.format(PRS[i,0]+(sphElements*(k)), j, PRS[i,4]))

## 8.6 Write data to be printed out by abaqus

In [15]:
#Write Outpur requests
fidab.write('*EL PRINT, POSITION=CENTROIDAL, ELSET=allEls \n')
fidab.write('S,\n')
fidab.write('*END STEP\n')
fidab.close()